In [1]:

import pandas as pd
import json

# Parse .json files from national security database
# cveFile should be the .json that comes from the NVD datbase. For example: nvdcve-1.1-2018.json
# newCSVname is optional. If newCSVname is passed, it should end in .csv
# NVDtoDF returns a dataframe
def NVDtoDF(cveFile, newCSVname=None):

    #
    with open(cveFile, "r") as read_file:
        data = json.load(read_file)
    df = pd.DataFrame(data['CVE_Items'])

    # get number of vulnerabilities in cveFile
    cveCount = df['cve'].count()

    # these lists will populate be used to create the dataframe
    IDarray = []
    descriptionArray = []
    attackVectorArr = []
    baseScore3Arr = []
    baseSeverity3Arr = []
    exploitablityScore3Arr = []
    impactScore3Arr = []
    baseScore2Arr = []
    severity2Arr = []
    exploitablityScore2Arr = []
    cveCount = df.cve.count()
    unableToFill = "couldNotFill"

    # populate each list
    for i in range(0, cveCount):

        IDarray.append(data['CVE_Items'][i]['cve']['CVE_data_meta']['ID'])
        descriptionArray.append(data['CVE_Items'][i]['cve']['description']['description_data'][0]['value'])
        if ('baseMetricV3' in data['CVE_Items'][i]['impact']):
            attackVectorArr.append(data['CVE_Items'][i]['impact']['baseMetricV3']['cvssV3']['attackVector'])
            baseSeverity3Arr.append(data['CVE_Items'][i]['impact']['baseMetricV3']['cvssV3']['baseSeverity'])
            baseScore3Arr.append(data['CVE_Items'][i]['impact']['baseMetricV3']['cvssV3']['baseScore'])
            exploitablityScore3Arr.append(data['CVE_Items'][i]['impact']['baseMetricV3']['exploitabilityScore'])
            impactScore3Arr.append(data['CVE_Items'][i]['impact']['baseMetricV3']['impactScore'])
        else:
            attackVectorArr.append(unableToFill)
            baseSeverity3Arr.append(unableToFill)
            baseScore3Arr.append(unableToFill)
            exploitablityScore3Arr.append(unableToFill)
            impactScore3Arr.append(unableToFill)
        if ('baseMetricV2' in data['CVE_Items'][i]['impact']):
            baseScore2Arr.append(data['CVE_Items'][i]['impact']['baseMetricV2']['cvssV2']['baseScore'])
            severity2Arr.append(data['CVE_Items'][i]['impact']['baseMetricV2']['severity'])
            exploitablityScore2Arr.append(data['CVE_Items'][i]['impact']['baseMetricV2']['exploitabilityScore'])
        else:
            baseScore2Arr.append(unableToFill)
            severity2Arr.append(unableToFill)
            exploitablityScore2Arr.append(unableToFill)

    # convert each populated list to a pandas/numpy Series
    IDs = pd.Series(IDarray)
    descriptions = pd.Series(descriptionArray)
    attackVector = pd.Series(attackVectorArr)
    baseScore3 = pd.Series(baseScore3Arr)
    baseSeverity3 = pd.Series(baseSeverity3Arr)
    exploitablityScore3 = pd.Series(exploitablityScore3Arr)
    impactScore3 = pd.Series(impactScore3Arr)
    baseScore2 = pd.Series(baseScore2Arr)
    severity2 = pd.Series(severity2Arr)
    exploitablityScore2 = pd.Series(exploitablityScore2Arr)

    # create each column
    df['CVE_ID'] = IDs
    df['description'] = descriptions
    df['attack_vector'] = attackVector
    df['baseScore_V3'] = baseScore3
    df['baseSeverity_V3'] = baseSeverity3
    df['exploitablityScore_V3'] = exploitablityScore3
    df['impactScore_V3'] = impactScore3
    df['baseScore_V2'] = baseScore2
    df['severity_V2'] = severity2
    df['exploitablityScore_V2'] = exploitablityScore2

    # Making descriptions lowercase makes finding technology keywords easier.
    df['description'] = df['description'].str.lower()

    CVEorg = df
    # drop extra columns
    CVEorg = CVEorg.drop(columns=['configurations', 'cve', 'impact', 'lastModifiedDate', 'publishedDate'])

    # write to newCSVname.csv in current directory
    if newCSVname:
        CVEorg.to_csv('%s.csv' % newCSVname, index=False)

    return CVEorg

# Processes dataframe from NVDtoDF, along with a list of technology keywords into a text file that will be
# input to the attacker instance.
# dframe is the processed CVE dataframe
# keyword_list is a list of lowercase strings of technlogies
# new_file_name is the name of the file to write to
def get_input_txt(dframe, keyword_list, new_file_name):
    temp_df = dframe.copy()

    temp_df['description'] = temp_df['description'].str.lower()

    # Create boolian for each keyword that specifies whether it is found in the vulnerability description
    for k in keyword_list:
        temp_df[k] = temp_df['description'].str.contains(pat=k)

    # Open/create txt file
    output = open('%s.txt' % new_file_name, "w")
    output.seek(0)
    output.truncate()
    for k in keyword_list:
        # Dataframe of only CVEs with keyword k in the description
        k_df = temp_df[temp_df[k] == True]
        cve_string = ""
        score_string = ""

        for index, row in k_df.iterrows():
            # Row of CVE-IDs seperated by '|'s
            cve_string += (str(row['CVE_ID']) + '|')
            # Row of sets of base score, exploitability score, impact score seperated by whitespace
            score_string += (str(row['baseScore_V3']) + ',' + str(row['exploitablityScore_V3']) + \
                             ',' + str(row['impactScore_V3']) + ' ')
        cve_string = cve_string[:-1] # take extra '|' from end
        score_string = score_string[:-1] # take extra whitespace from end

        # write to file
        output.write("keyword: ")
        output.write(str(k))
        output.write("\n")
        output.write("CVE count: ")
        output.write(str(k_df['CVE_ID'].count()))
        output.write("\n")
        output.write(cve_string)
        output.write("\n")
        output.write("\n")
        output.write(score_string)
        output.write("\n")
        output.write("\n")

    output.close()


# Combines NVDtoDF and get_input_text functions to take a .json from the NVD database
def NVDtoTXT(CVEfile, keyword_list, new_file_name):
    myDF = NVDtoDF(CVEfile, False)
    get_input_txt(myDF, keyword_list, new_file_name)








Below we can see the first few rows of the dataframe output by the NVDtoDF function.

In [2]:
df_2018 = NVDtoDF("nvdcve-1.1-2018.json")
NVDtoDF("nvdcve-1.1-2018.json").head()

,CVE_ID,description,attack_vector,baseScore_V3,baseSeverity_V3,exploitablityScore_V3,impactScore_V3,baseScore_V2,severity_V2,exploitablityScore_V2
0,CVE-2018-0001,"a remote, unauthenticated attacker may be able...",NETWORK,9.8,CRITICAL,3.9,5.9,7.5,HIGH,10
1,CVE-2018-0002,on srx series and mx series devices with a ser...,NETWORK,5.9,MEDIUM,2.2,3.6,4.3,MEDIUM,8.6
2,CVE-2018-0003,a specially crafted mpls packet received or pr...,ADJACENT_NETWORK,6.5,MEDIUM,2.8,3.6,6.1,MEDIUM,6.5
3,CVE-2018-0004,a sustained sequence of different types of nor...,NETWORK,6.5,MEDIUM,2.8,3.6,7.1,HIGH,8.6
4,CVE-2018-0005,qfx and ex series switches configured to drop ...,ADJACENT_NETWORK,8.8,HIGH,2.8,5.9,5.8,MEDIUM,6.5


The list 'key' below is an example list of keywords that we could look for in CVE descriptions. This example list was compiled by choosing relevant words from the most commonly occuring words in the CVE descriptions. The best way to generate such a list is still an open question. Right now we are considering better ways to generate such a list, such as using keywords from a port scan of a targeted machine or calculating the inverse document frequency of words in each CVE description to identify relevant words in the National Vulnerability Database.

In [4]:
# This is a list of the most commonly occuring 400 words in CVE descriptions.
from collections import Counter
Counter(" ".join(df_2018["description"]).split()).most_common(400)

[('the', 28636),
 ('to', 24171),
 ('in', 20953),
 ('a', 19355),
 ('of', 14439),
 ('and', 12847),
 ('an', 11568),
 ('vulnerability', 9483),
 ('this', 8669),
 ('is', 6402),
 ('attacker', 5514),
 ('allows', 5215),
 ('that', 5010),
 ('via', 4971),
 ('can', 4041),
 ('could', 3974),
 ('or', 3957),
 ('for', 3953),
 ('was', 3926),
 ('remote', 3872),
 ('with', 3718),
 ('before', 3628),
 ('on', 3395),
 ('access', 3336),
 ('code', 3283),
 ('by', 3280),
 ('versions', 3242),
 ('arbitrary', 3170),
 ('not', 3147),
 ('user', 3002),
 ('issue', 2917),
 ('windows', 2595),
 ('allow', 2585),
 ('attackers', 2464),
 ('from', 2295),
 ('oracle', 2293),
 ('has', 2279),
 ('affected', 2245),
 ('prior', 2160),
 ('be', 2123),
 ('server', 2123),
 ('are', 2091),
 ('as', 2046),
 ('version', 2009),
 ('crafted', 1983),
 ('sd', 1962),
 ('cause', 1952),
 ('successful', 1929),
 ('execute', 1914),
 ('**', 1888),
 ('service', 1880),
 ('discovered', 1801),
 ('exists', 1779),
 ('exploit', 1756),
 ('candidate', 1707),
 ('denial

In [5]:
key = ['python', 'php', 'mysql', 'postgresql','sql', 'xxs', 'windows', 'oracle', \
            'java ', 'html', 'http', 'injection', 'cross-site', 'javascript','linux', \
            'xml', 'url', 'cross-site']

In [6]:
NVDtoTXT("nvdcve-1.1-2018.json", key, "output2018-001")

Below is the content of the file produced by the call of NVDtoTXT. There are four lines associated with each word in our keyword list. The first line lists the keyword. The second line list the number of CVE descriptions that keyword appeared in. The Third is a list of all of the CVE_IDs (seperated by '|') that contain the keyword. The fourth is a list of base score, exploitability score, impact score (seperated by whitespace) of the CVE listed in the line above.

In [7]:
example_file = open("output2018-001.txt")
example_text = example_file.read()
print(example_text)
example_file.close()

keyword: python
CVE count: 31
CVE-2018-0015|CVE-2018-0023|CVE-2018-1000030|CVE-2018-1000117|CVE-2018-1000802|CVE-2018-1000807|CVE-2018-1000808|CVE-2018-1060|CVE-2018-1061|CVE-2018-10903|CVE-2018-12175|CVE-2018-14572|CVE-2018-14647|CVE-2018-14649|CVE-2018-15747|CVE-2018-16168|CVE-2018-16858|CVE-2018-17175|CVE-2018-18074|CVE-2018-19646|CVE-2018-20061|CVE-2018-20325|CVE-2018-20406|CVE-2018-20852|CVE-2018-2753|CVE-2018-3650|CVE-2018-5773|CVE-2018-6012|CVE-2018-6353|CVE-2018-6461|CVE-2018-7889

7.5,1.6,5.9 5.5,1.8,3.6 8.1,2.2,5.9 6.7,0.8,5.9 9.8,3.9,5.9 8.1,2.2,5.9 5.9,2.2,3.6 7.5,3.9,3.6 7.5,3.9,3.6 7.5,3.9,3.6 7.8,1.8,5.9 7.8,1.8,5.9 7.5,3.9,3.6 9.8,3.9,5.9 9.8,3.9,5.9 9.8,3.9,5.9 9.8,3.9,5.9 5.3,3.9,1.4 9.8,3.9,5.9 9.8,3.9,5.9 7.5,3.9,3.6 9.8,3.9,5.9 7.5,3.9,3.6 5.3,3.9,1.4 6.0,0.8,5.2 7.8,1.8,5.9 6.1,2.8,2.7 9.8,3.9,5.9 7.8,1.8,5.9 7.8,1.8,5.9 7.8,1.8,5.9

keyword: php
CVE count: 1511
CVE-2018-0001|CVE-2018-0535|CVE-2018-0568|CVE-2018-0645|CVE-2018-0658|CVE-2018-1000019|CVE-2018-1000020